In [ ]:
# Python Standard Import Statements
# .............................
#%matplotlib inline
import matplotlib; matplotlib.use('agg')

import netCDF4
import shapely
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.odr as odr
import scipy.signal as signal
import scipy.integrate as integrate
import importlib
import datetime as dt
import mpl_toolkits.basemap as Basemap

# Imports for Polygon Routines
# ..................................

from netCDF4 import Dataset
from shapely import geometry
from shapely import ops
from decimal import Decimal
# .............................

def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print ("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print ('\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr)))
        except KeyError:
            print ("\t\tWARNING: %s does not contain variable attributes") % key

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print ("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print ('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print ("NetCDF dimension information:")
        for dim in nc_dims:
            print ("\tName:", dim) 
            print ("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print ("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print ('\tName:', var)
                print ("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print ("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

# Defined Functions to be used in script below
# .................................................

# Function for grabbing coordinates 2PVU contour path
def get_path_coord(contour):
    paths = []
    paths = contour.collections[0].get_paths()
    n = len(paths)
    xy = []
    length = []
    for i in np.arange(0,n,1):
        length.append(len(paths[i]))
    r = np.argmax(length) # gives index of largest contour path
    
    paths1 = paths[r]
    xy = paths1.vertices

    for i in range(len(xy)):
        a = xy[:,0]
        b = xy[:,1]
    return a, b
# ..................................
# Function for finding the index of the closes value
# Input Parameter
# ..................
# A: Array we want the index from
#target: The array we would like to know the indices for

def getnearpos(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx   

# ..................................
# Python code to find longest running 
# sequence of positive integers.

# Problem with code. Doesn't seem to see last large positive consec
# group. Pad data at the end with negative values.
 
def getLongestSeq(a, n):
    maxIdx = 0
    maxLen = 0
    currLen = 0
    currIdx = 0
    
    #check to see if data needs padding at the end
    for k in range(n):
        if a[k] > 0:
            currLen +=1
 
            # New sequence, store
            # beginning index.
            if currLen == 1:
                currIdx = k
        else:
            if currLen > maxLen:
                maxLen = currLen
                maxIdx = currIdx
            currLen = 0
             
    if maxLen > 0:
        print('Index : ',maxIdx,',Length : ',maxLen,)
    else:
        print("No positive sequence detected.")
    return maxIdx, maxLen

# ...............................
# Function for collecting start and end points of contour
# Input Parameters
# ...........................
# strt_idx: index of first point of pv gradient reversal
# maxl: length of pv gradient reversal
# last: index of last value of array

def start_end_contour(fidx,maxl,last):    
    md_idx = fidx
    end_idx = md_idx + maxl
    strt_idx = md_idx - maxl
    
    if end_idx > last:
        if strt_idx < 0:
            strt_idx1 = 0
            return strt_idx1, md_idx,last
        else:
            return strt_idx, md_idx, last
    else:
        if strt_idx < 0:
            strt_idx1 = 0
            return strt_idx1, md_idx, end_idx
        else:
            return strt_idx, md_idx, end_idx

# ......................................        
# Function for retrieving path of contour and create Polygon
# Algorithm for retrieving 2-PVU contour paths for each 
# Retrieve 2-PVU contour path.

def retrieve_path(contour):
    paths = []
    paths = contour.collections[0].get_paths()
    n = len(paths)
    xy = []
    length = []
    for i in np.arange(0,n,1):
        length.append(len(paths[i]))
    r = np.argmax(length) # gives index of largest contour path
    
    paths1 = paths[r]
    xy = paths1.vertices

    for i in range(len(xy)):
        a = xy[:,0]
        b = xy[:,1]
    poly = geometry.Polygon([(i[0],i[1]) for i in zip(a,b)])
    
    # find intersection between line and contour segment
    
    return poly

# .............................
# Function for choosing the start point of the PV streamer segment
# Code tests whether p2gradx2 is positive at positive p2grady2
# Input Variables:
# firstx, firsty: index of first value indicated by getLongSeq
# lengthx, lengthy: length of consecutive group of positive values indicated by getLongSeq

# Output Variables:
# strt_idx, end-idx: start and end index bounds of PV streamer
# .........................................

def pvindex_bnds(datax,datay,firstx,firsty,lengthx,lengthy):
    idx1 = np.min([firstx,firsty])  # Idnetify range over which to search
    idx2 = np.max([firstx,firsty])+1
    irange = np.arange(idx1,idx2,1)

    for i in irange:     # Search along positive p2grady2 for index with positive p2gradx2
        if (datay[i]>0 and datax[i]>0):
            strt_idx = i
            pos_length = np.max([lengthx,lengthy])  # Find index detection length (pos_length x 2) of the PV streamer.
            end_idx = strt_idx + (pos_length*2)     # Find end index. 
            mdl_idx = strt_idx + np.min([lengthx,lengthy]) # Find middle index to be used to define PV streamer region.
            
            if end_idx >= len(p2grady2):            # Check whether end_idx is greater than length of contour path
                end_idx = len(p2grady2)-2
            else:
                end_idx = end_idx
            
            if mdl_idx >= len(p2grady2):            # Check whether mdl_idx is greater than length of contour path
                mdl_idx = len(p2grady2)-2
            else:
                mdl_idx = mdl_idx
            break
        else:
            strt_idx = float('NaN')
            mdl_idx = float('NaN')
            end_idx = float('NaN')
            pass
            
    return strt_idx, mdl_idx, end_idx

# ....................................
# Function
# Pad data with negatives at the end. Section tests whether or not the very last element is positive or negative,
# since getLongSec function doesn't "see" positive end values very well unless part of a significantly large
# sequence. If end value is positive (i.e. p2grady2[-1]>0), then test is padded with -1's.
# ...........................................................
# Store size of p2gradx2 and p2grady2

def data_pad(datax,datay):
    # Tests if padding needed
    if (datay[-1]>0 or datax[-1]>0):
        testy = np.copy(datay)
        testx = np.copy(datax)
        pad = np.repeat(-1,5)
        testy2 = np.hstack([testy,pad])
        testx2 = np.hstack([testx,pad])
    else:
        testy2 = np.copy(datay)
        testx2 = np.copy(datax)
    
    return testx2, testy2

# Function for calculating the fourier coefficents (Ao, Ak, and Bk),
# variance explained (Ck squared) and the sample frequencies. 
# Currently one-dimensional.
def fft_coeff_numpy(f,T):

# Input Parameters
# ....................
# f: n-dimensional data
# T: length of window/data

# Output Parameters
# ....................
# 

# Record length must be odd, so here we test whether the data length
# is even or odd. If it is odd, T remains. If it is even, T becomes T-1.
    if T % 2 == 0:
        T = T-1 # even, subtract 1
    else:
        T = T # odd, nothing changes
    

# Size of data. T must be odd. Therefore, t is also odd. But, N is even.
    dt = 1 # change to aquire different sample rate
    t= np.arange(0,T,dt)
    N = np.size(t)-1


# Define window size and frequencies
    freq = fft.fftfreq(T,dt)
    nyquist = N/(2*T)
    
# Calculate wavenumbers
    T2 = N/2
    k = np.arange(1,T2,1)
    
# Calculate fourier series
    y1 = (fft.fft(f[np.arange(0,t.size,1)]))/t.size
    y2 = fft.fft(f[np.arange(0,t.size,1)])

# Calculate fourier coefficients
    Ao = y1[0].real
    Ak = y1[1:-1].real
    Bk = -y1[1:-1].imag
    C = np.absolute(y1)

# Calculate fraction of variance explained and normalize
    pve = ((C**2)/np.sum(C**2))    # scale with 2 to obtain area under curve of 1.

    return y1, y2, freq, T2, nyquist, k, Ao, Ak, Bk, C, pve, T

# ..................................
# Function for bandpass filtering
# Source: Libby
# Currently set to use mean + first three harmonics
# Input Variables
# data: your time series or data
# fidx: index/indices of desired harmonics/frequencies
# ..................
# Output Variables
# lowpass-flt: time series of low-frequency portion of data
# highpass_flt: time series of high-frequency portion of data

# INCOMPLETE
def fft_filter1(transform):
    # Lowpass filter
    dtlow = np.copy(transform)
    dtlow[4:] = 0.
    #dtlow[0:3] = 0.
    
    lowpass_flt = np.real(fft.ifft(dtlow))
    
    # Highpass filter
    dthigh = np.copy(transform)
    dthigh[0:3:] = 0.
    #dthigh[-2::] = 0.
    
    highpass_flt = np.real(np.fft.ifft(dthigh))
    
    return lowpass_flt, highpass_flt

# NC file input

ncfile='/home/jjpjones/pvs_vws_indices/era5_pv350k_6hrly_v2.nc'
ncf = Dataset(ncfile,'r')
nc_attrs, nc_dims, nc_vars = ncdump(ncf)

# Extract data from NetCDF file
lat = ncf.variables['latitude'][:]
lon = ncf.variables['longitude'][:]
time = ncf.variables['time'][:]
pv = ncf.variables['pv'][:]

# List all times in file as datetime objects
#time2 = [int(i) for i in time]
#dt_time = [dt.date(1900,1,1) + dt.timedelta(hours=t) for t in time2]

clim0, clim6, clim12, clim18 = np.empty([73,144]),np.empty([73,144]),np.empty([73,144]),np.empty([73,144])
std0, std6, std12, std18 = np.empty([73,144]),np.empty([73,144]),np.empty([73,144]),np.empty([73,144])

for j in np.arange(0,144,1):
    for i in np.arange(0,73,1):
        clim_range = pv[np.arange(0,58800,1),i,j]
        dates = pd.date_range('1979-01-01','2019-04-01',freq='6H')
        clim_frame = pd.Series(clim_range,index=dates[0:58800])
        clim = clim_frame.groupby(clim_frame.index.hour).mean()
        clim2 = np.array(clim.values)
        pvstd = clim_frame.groupby(clim_frame.index.hour).std()
        pvstd2 = np.array(pvstd.values)
        
        clim0[i,j], clim6[i,j], clim12[i,j], clim18[i,j] = clim2[0],clim2[1],clim2[2],clim2[3]
        std0[i,j], std6[i,j], std12[i,j], std18[i,j] = pvstd2[0],pvstd2[1],pvstd2[2],pvstd2[3]
        
# Set up empty arrays
pvsi_full = []
strt1, mid1, end1 = [],[],[]
strt2, mid2, end2 = [],[],[]

for q in np.arange(0,58800,1):
    # Define field and calculate meridional pv gradient field
    pv_field = pv[q,0:73,0:143]
    pv_diffx = np.gradient(pv_field, axis=0)
    pv_diffy = np.gradient(pv_field, axis =1)
    
    # Plot 2PVU contour and retrieve coordinates along path
    cs1 = []
    cs1 = plt.contour(lon[104:143],lat[12:34],pv_field[12:34,104:143],levels=[2e-6])
    a, b = get_path_coord(cs1)
    # ...........................................
    
    # Retrieve indices for contour coordinates
    
    idxlt, idxln = [], []
    for r in b:
        idxlt.append(getnearpos(lat,r))
    
    for r in a:
        idxln.append(getnearpos(lon,r))
    
    idxlt = np.flipud(np.array(idxlt)) # convert to array. Update v2: position indices flipped to move from 
    idxln = np.flipud(np.array(idxln)) # left to right  
    # ...........................................
    
    # Retrieve pv gradient values corresponding to 2PVU contour
    # Added array for zonal pv gradient field

    p2gradx, p2grady = [],[]
    p2gradx.append(pv_diffx[idxlt,idxln])
    p2grady.append(pv_diffy[idxlt,idxln])
    
    # Convert lists to arrays
    p2gradx = np.array(p2gradx)
    p2grady = np.array(p2grady)
    
    # Change from hstack to vstack
    p2gradx2 = p2gradx[0,:]
    p2grady2 = p2grady[0,:]
    
    # ...........................................
    # Update v2: Data padding moved to user-defined function data_pad. pvindex_bnds replaces
    # start_end_contour.
    # Find largest group of consecutive positive values
    # Create temporary array (test) and pad data with negatives 
    # at the end.
    
    # Section tests whether or not the very last element is positive or negative
    # since getLongSec function doesn't "see" positive end values very well. If 
    # end value is positive (i.e. p2grad2[-1]>0), then test is padded with -1's.
    testx, testy = data_pad(p2gradx2,p2grady2) 
    
    
    # Check for a large group of consecutive positive integers in zonal meridional pv gradient
    #test
    pidxy1, plenty1 = getLongestSeq(testy,len(testy))
    pidxx1, plentx1 = getLongestSeq(testx,len(testx))
   
    if (plenty1 !=0 and plentx1 !=0):
        strt1, mid1, end1 = pvindex_bnds(testx,testy,pidxx1,pidxy1,plentx1,plenty1)
    else:
        print('Step pvindex_bnds: No primary PV detected at timestep: '+ str(q))
        strt1 = float('NaN')
        mid1 = float('NaN')
        end1 = float('NaN')
        pass

    # ...........................................
    # Check for second major pv reversal
    # Check for a second major group of consecutive positive integers
    # Update v2
    
    if np.isnan(strt1) == False:
        
        # Define start, middle, and end points for
        # First PV
        lts1, lns1 = idxlt[strt1], idxln[strt1]
        ltm1, lnm1 = idxlt[mid1], idxln[mid1]
        lte1, lne1 = idxlt[end1], idxln[end1]
        
        # Create box around detected PV. Find box edges of pv streamer region.
        #flat1, flat2, llat1, llat2 = [], [], [], []
        flat1, llat1 = np.min([lts1,ltm1,lte1])-4,np.max([lts1,ltm1,lte1])+4
        flon1, llon1 = np.min([lns1,lnm1,lne1])-4,np.max([lns1,lnm1,lne1])+4
        
        dims1 = np.size(lon[flon1:llon1])*np.size(lat[flat1:llat1])
        
        # Check for a second major group of consecutive positive integers by
        # removing first group (replaced with -1's)
        zero_pad1 = np.abs(end1 - strt1)
        y_remaining = np.copy(testy)
        y_remaining[strt1:end1] = np.repeat(-1,zero_pad1)

        x_remaining = np.copy(testx)
        x_remaining[strt1:end1] = np.repeat(-1,zero_pad1)

        pidxy2, plenty2 = getLongestSeq(y_remaining,len(y_remaining))
        pidxx2, plentx2 = getLongestSeq(x_remaining,len(x_remaining))
    
        # Check to make sure that lengths of group are greater than 0. 
        if (plenty2 != 0 and plentx2 != 0):
            strt2, mid2, end2 = pvindex_bnds(x_remaining,y_remaining,pidxx2,pidxy2,plentx2,plenty2)
            
            if np.isnan(strt2) == False:
                # Position of start, middle, and end index of second PV
                lts2, lns2 = idxlt[strt2], idxln[strt2]
                ltm2, lnm2 = idxlt[mid2], idxln[mid2]
                lte2, lne2 = idxlt[end2], idxln[end2]
            
                # Create box around second detected PV. 
                flat2, llat2 = np.min([lts2,ltm2,lte2])-4, np.max([lts2,ltm2,lte2])+4
                flon2, llon2 = np.min([lns2,lnm2,lne2])-4, np.max([lns2,lnm2,lne2])+4
            
                # Calculate box dimensions.
                dims2 = np.size(lon[flon2:llon2])*np.size(lat[flat2:llat2])
            else:
                flat2, llat2 = float('NaN'), float('NaN')
                flon2, llon2 = float('NaN'), float('NaN')
                dims2=0
        else:
            flat2, llat2 = float('NaN'), float('NaN')
            flon2, llon2 = float('NaN'), float('NaN')
            dims2=0
    else:
        flat1, llat1 = float('NaN'), float('NaN')
        flon1, llon1 = float('NaN'), float('NaN')
        
        flat2, llat2 = float('NaN'), float('NaN')
        flon2, llon2 = float('NaN'), float('NaN')
        
        dims1 = 0
        dims2 = 0
        pass
    
    # ................................
    
    if np.any(np.isnan([flat1,llat1,flon1,llon1])) == True:
        pvpoly1 = geometry.Polygon()
    else:
        if not np.size(pv_field[flat1:llat1,flon1:llon1]) == dims1:
            pvpoly1 = geometry.Polygon()
        else:
            plt.figure(1)
            plt.clf()
            cnt1 = plt.contour(lon[flon1:llon1],lat[flat1:llat1],pv_field[flat1:llat1,flon1:llon1],levels=[2e-6])
            pvpoly1 = retrieve_path(cnt1)
            
    if np.any(np.isnan([flat2,llat2,flon2,llon2])) == True:
        pvpoly2 = geometry.Polygon()
    else:
        if not np.size(pv_field[flat2:llat2,flon2:llon2]) == dims2:
            pvpoly2 = geometry.Polygon() 
        else:
            plt.figure(2)
            plt.clf()
            cnt2 = plt.contour(lon[flon2:llon2],lat[flat2:llat2],pv_field[flat2:llat2,flon2:llon2],levels=[2e-6])
            plt.plot()
            pvpoly2 = retrieve_path(cnt2)
        
    # .................................................
    # Draw latitudes and longitudes for region
    lt = lat[20:28]
    ln = lon[104:143]

    ln_st, ln_ed = [], []
    lt_st, lt_ed = [], []

    # latitude
    for i in lt:
        lidx1 = len(ln)-1
        lt_st.append((ln[0],i))
        lt_ed.append((ln[lidx1],i))

    # longitudes
    for i in ln:
        lidx2 = len(lt)-1
        ln_st.append((i,lt[0]))
        ln_ed.append((i,lt[lidx2]))
    # ..................................
    # Create multiple Shapely lines and store into one string
    # Create coords lists
    lat_coords = [[x,y] for x,y in zip(lt_st,lt_ed)]
    lon_coords = [[x,y] for x,y in zip(ln_st,ln_ed)]

    # String for multiple lines
    lt_lines = geometry.MultiLineString(lat_coords)
    ln_lines = geometry.MultiLineString(lon_coords)
    
    # Intersection between latitudes and longitudes
    lat_lon_intersec = lt_lines.intersection(ln_lines)
    
    # Find intersection between lat/lon intersections 
    # and pv polygon area
    
    if not (pvpoly1.is_empty) == True: 
        try:
            FINAL_intersec1 = lat_lon_intersec.intersection(pvpoly1.buffer(1))
        except Exception:
            pass
        pvs_coordinates1 = np.array(geometry.shape(geometry.mapping(FINAL_intersec1)))
    else:
        print('Step FINAL_intersec1: No polygon detected.')
        pvs_coordinates1 = np.empty(shape=(0,0))
   
    if not (pvpoly2.is_empty) == True:
        try:
            FINAL_intersec2 = lat_lon_intersec.intersection(pvpoly2.buffer(1))
        except Exception:
            pass
        pvs_coordinates2 = np.array(geometry.shape(geometry.mapping(FINAL_intersec2)))
    else:
        print('Step FINAL_intersec2: No 2nd polygon detected.')
        pvs_coordinates2 = np.empty(shape=(0,0))
        
        
    # Find coordinates
    # Largest PV streamer
    if pvs_coordinates1.size==0:
        xp1, yp1 = [], []
        pv_anom1 = float('NaN')
        pass     
    elif pvs_coordinates1.size > 2:
        xp1 = pvs_coordinates1[:,0]
        yp1 = pvs_coordinates1[:,1]
    else:
        xp1 = pvs_coordinates1[0]
        yp1 = pvs_coordinates1[1]
        
    # Second Largest PV streamer
    if pvs_coordinates2.size==0:
        xp2, yp2 = [], []
        pv_anom2 = float('NaN')
        pass     
    elif pvs_coordinates2.size > 2:
        xp2 = pvs_coordinates2[:,0]
        yp2 = pvs_coordinates2[:,1]
    else:
        xp2 = pvs_coordinates2[0]
        yp2 = pvs_coordinates2[1]
    
    # Find indices for coordinates
    x_idx1, y_idx1 = [], []
    x_idx2, y_idx2 = [], []

    # x values
    if isinstance(xp1,list)==True:
        if len(xp1) ==0:
            pass
        elif len(xp1)>1:
            for i in xp1:
                x_idx1.append(getnearpos(lon,i))
        else:
            x_idx1.append(getnearpos(lon,xp1))
    else:
        if xp1.size ==0:
            pass
        elif xp1.size>1:
            for i in xp1:
                x_idx1.append(getnearpos(lon,i))
        else:
            x_idx1.append(getnearpos(lon,xp1)) 
    
    if isinstance(xp2,list) == True:
        if len(xp2) ==0:
            pass
        elif len(xp2)>1:
            for i in xp2:
                x_idx2.append(getnearpos(lon,i))
        else:
            x_idx2.append(getnearpos(lon,xp2))
    else:
        if xp2.size ==0:
            pass
        elif xp2.size>1:
            for i in xp2:
                x_idx2.append(getnearpos(lon,i))
        else:
            x_idx2.append(getnearpos(lon,xp2)) 
        
    # y values    
    if isinstance(yp1,list) == True:
        if len(yp1) ==0:
            pass
        elif len(yp1)>1:
            for i in yp1:
                y_idx1.append(getnearpos(lat,i))
        else:
            y_idx1.append(getnearpos(lat,yp1))
    else:
        if yp1.size ==0:
            pass
        elif yp1.size>1:
            for i in yp1:
                y_idx1.append(getnearpos(lat,i))
        else:
            y_idx1.append(getnearpos(lat,yp1)) 
    
    if isinstance(yp2,list) == True:
        if len(yp2) ==0:
            pass
        elif len(yp2)>1:
            for i in yp2:
                y_idx2.append(getnearpos(lat,i))
        else:
            y_idx2.append(getnearpos(lat,yp2))
    else:
        if yp2.size ==0:
            pass
        elif yp2.size>1:
            for i in yp2:
                y_idx2.append(getnearpos(lat,i))
        else:
            y_idx2.append(getnearpos(lat,xp2)) 
        
    # .......................................
    # Retrieve pv values (OBSOLETE)
    #pvs_area_pv1, pvs_area_pv2 = [], []
    #pvs_area_pv1 = [pv_field[y,x] for y,x in zip(y_idx1,x_idx1)]
    #pvs_area_pv2 = [pv_field[y,x] for y,x in zip(y_idx2,x_idx2)]
    
    # Find standardized anomalies of grid points within polygon
    if dates[q].hour == 0:
        if (len(y_idx1)!=0 and len(x_idx1)!=0):
            for y,x in zip(y_idx1, x_idx1):
                pv_anomaly1 = ((pv_field[y,x]-clim0[y,x])/std0[y,x])
        else:
            pv_anomaly1 = np.repeat(float('NaN'),5)
    elif dates[q].hour == 6:
        if (len(y_idx1)!=0 and len(x_idx1)!=0):
            for y,x in zip(y_idx1, x_idx1):
                pv_anomaly1 = ((pv_field[y,x]-clim6[y,x])/std6[y,x])
        else:
            pv_anomaly1 = np.repeat(float('NaN'),5)
    elif dates[q].hour == 12:
        if (len(y_idx1)!=0 and len(x_idx1)!=0):
            for y,x in zip(y_idx1, x_idx1):
                pv_anomaly1 = ((pv_field[y,x]-clim12[y,x])/std12[y,x])
        else:
            pv_anomaly1 = np.repeat(float('NaN'),5)
    else:
        if (len(y_idx1)!=0 and len(x_idx1)!=0):
            for y,x in zip(y_idx1, x_idx1):
                pv_anomaly1 = ((pv_field[y,x]-clim18[y,x])/std18[y,x])
        else:
            pv_anomaly1 = np.repeat(float('NaN'),5)
             
    if dates[q].hour == 0:
        if (len(y_idx2)!=0 and len(x_idx2)!=0):
            for y,x in zip(y_idx2, x_idx2):
                pv_anomaly2 = ((pv_field[y,x]-clim0[y,x])/std0[y,x])
        else:
            pv_anomaly2 = np.repeat(float('NaN'),5)
    elif dates[q].hour == 6:
        if (len(y_idx2)!=0 and len(x_idx2)!=0):
            for y,x in zip(y_idx2, x_idx2):
                pv_anomaly2 = ((pv_field[y,x]-clim6[y,x])/std6[y,x])
        else:
            pv_anomaly2 = np.repeat(float('NaN'),5)
    elif dates[q].hour == 12:
        if (len(y_idx2)!=0 and len(x_idx2)!=0):
            for y,x in zip(y_idx2, x_idx2):
                pv_anomaly2 = ((pv_field[y,x]-clim12[y,x])/std12[y,x])
        else:
            pv_anomaly2 = np.repeat(float('NaN'),5)
    else:
        if (len(y_idx2)!=0 and len(x_idx2)!=0):
            for y,x in zip(y_idx2, x_idx2):
                pv_anomaly2 = ((pv_field[y,x]-clim18[y,x])/std18[y,x])
        else:
            pv_anomaly2 = np.repeat(float('NaN'),5)
    # ......................................................................
    
    # Test for NaNs. I remove NaNs here, why????
    if pv_anomaly1.size>1:
        if np.isnan(pv_anomaly1.any())== True:
            pv_anomaly1[np.isnan(pv_anomaly1)]=0
        elif np.isnan(pv_anomaly1.all())== True:
            pv_anomaly1 = 0
        else:
            pv_anomaly1 = pv_anomaly1
    else:
        if np.isnan(pv_anomaly1.any())== True:
            pv_anomaly1[np.isnan(pv_anomaly1)]=0
        elif np.isnan(pv_anomaly1)== True:
            pv_anomaly1 = 0
        else:
            pv_anomaly1 = pv_anomaly1 
    pv_anom1 = np.sum(np.float64(pv_anomaly1))
    
    if pv_anomaly2.size>1:
        if np.isnan(pv_anomaly2.any())== True:
            pv_anomaly2[np.isnan(pv_anomaly2)]=0
        elif np.isnan(pv_anomaly2.all())== True:
            pv_anomaly2 = 0
        else:
            pv_anomaly2 = pv_anomaly2
    else:
        if np.isnan(pv_anomaly2.any())== True:
            pv_anomaly2[np.isnan(pv_anomaly2)]=0
        elif np.isnan(pv_anomaly2)== True:
            pv_anomaly2 = 0
        else:
            pv_anomaly2 = pv_anomaly2
    pv_anom2 = np.sum(np.float64(pv_anomaly2))        
        
    # Sum pv anomaly over streamers to obtain 6hrly activity    
    daily_total = np.nansum([pv_anom1, pv_anom2])
    
    # Append to daily index
    pvsi_full.append(daily_total)
    print('Field at ', q, 'is complete.')
    
subdaily_pvsi = np.array(pvsi_full)
np.savetxt('/home/jjpjones/pvs_vws_indices/pvsi_full_6hrly_updates.txt',subdaily_pvsi,delimiter=' ')